# FUNCTIONAL CHARACTERIZATION TABLE

## RegulomeDB

In [1]:
library (tidyr)
library(dplyr)
library(data.table)

#Load the raw data
regulome <- fread('/home/mentoraje/PAEs/SNProject/RegulomeDB/TSTFF344324.tsv', sep = '\t')

#Change the column name from score to Regulome Score for better understanding
colnames(regulome)[12] <- 'RegulomeScore'

#Add the numeric impact column called RegulomeImportance
regulome <-  regulome %>% mutate(RegulomeImportance = case_when(
  RegulomeScore %in% '1a' ~ 1,
  RegulomeScore %in% '1b' ~ 2,
  RegulomeScore %in% '1c' ~ 3,
  RegulomeScore %in% '1d' ~ 4,
  RegulomeScore %in% '1e' ~ 5,
  RegulomeScore %in% '1f' ~ 6,
  RegulomeScore %in% '2a' ~ 7,
  RegulomeScore %in% '2b' ~ 8,
  RegulomeScore %in% '2c' ~ 9,
  RegulomeScore %in% '3a' ~ 10,
  RegulomeScore %in% '3b' ~ 11,
  RegulomeScore %in% '4' ~ 12,
  RegulomeScore %in% '5' ~ 13,
  RegulomeScore %in% '6' ~ 14,
  RegulomeScore %in% '7' ~ 15
  ))

#For each cromosome write a file with the data. Only rsid, RegulomeScore and RegulomeImportance columns
CHR <- c(1:22)
for (chr in CHR){
  print(paste0('-->',chr))
  regulome_fltr <- regulome %>% select(rsid, RegulomeScore, RegulomeImportance) %>% filter(regulome$chrom == paste0('chr',chr))
  
  fwrite(regulome_fltr, file = paste0('/home/mentoraje/PAEs/SNProject/RegulomeDB/Resultats/RegulomeChr',chr,'.txt'), sep = '\t')
  
}


SyntaxError: invalid syntax (<ipython-input-1-0f3d549778de>, line 7)

## ClinVar 

In [ ]:
library (tidyr)
library(dplyr)
library(data.table)

#Load the raw file
clinvar <- fread('/home/mentoraje/PAEs/SNProject/ClinVar/variant_summary_new.txt', header = TRUE)

#Change the column name to rsid for future merging
colnames(clinvar)[10] <- 'rsid'

#Remove duplicated SNPs, those that aren't from GRCh37 assembly and those that don't have rsid. 
clinvarGRch37 <- clinvar %>% 
  distinct(rsid, .keep_all = TRUE) %>% 
  filter(Assembly == 'GRCh37', rsid != -1)

#Add the 'rs' before the id number for future merging
clinvarGRch37$rsid <- c(paste0('rs',clinvarGRch37$rsid))

#Select the columns we are interested in
clinvar_fltr <- clinvarGRch37 %>% select(rsid, Chromosome, 
                                         ClinicalSignificance)

#Separate the Clinical significance values, from the SNPs that have more than one, in different rows 
clinvar_fltr <- separate_rows(clinvar_fltr, ClinicalSignificance, sep = ', ')

#Add a new column with the numeric impact (ClinicalImpact) of Clinical Significance 
clinvar_fltr <- clinvar_fltr %>% mutate(ClinicalImpact = case_when(
  ClinicalSignificance %in% 'Pathogenic' ~ 1,
  ClinicalSignificance %in% 'Pathogenic/Likely pathogenic' ~ 2,
  ClinicalSignificance %in% 'Likely pathogenic' ~ 3,
  ClinicalSignificance %in% 'drug response' ~ 4,
  ClinicalSignificance %in% 'association' ~ 5, 
  ClinicalSignificance %in% 'risk factor' ~ 6,
  ClinicalSignificance %in% 'Affects' ~ 7, 
  ClinicalSignificance %in% 'other' ~ 8,
  ClinicalSignificance %in% 'Conflicting interpretations of pathogenicity' ~ 9,
  ClinicalSignificance %in% 'Conflicting interpretations from submitters' ~ 10,
  ClinicalSignificance %in% 'Uncertain significance' ~ 11,
  ClinicalSignificance %in% 'no interpretation for the single variant' ~ 12,
  ClinicalSignificance %in% 'not provided' ~ 13,
  ClinicalSignificance %in% 'association not found' ~ 14,
  ClinicalSignificance %in% 'protective' ~ 15,
  ClinicalSignificance %in% 'Likely benign' ~ 16,
  ClinicalSignificance %in% 'Benign/Likely benign' ~ 17,
  ClinicalSignificance %in% 'Benign' ~ 18,
  
))

#Regroup the rows by rsid and create a new column with maximum Clinical Impact (most severe)
clinvar_def <- clinvar_fltr %>%
  group_by(rsid) %>%
  summarise(Chr = max(Chromosome),
            ClinicalSignificances = toString(ClinicalSignificance),
            ClinicaImpacts = toString(ClinicalImpact),
            MaxClinicalImpact = min(ClinicalImpact)) %>%
  ungroup() 

CHR <- c(1:22, 'X')

#Write the data in a different file for each chromosome
for (chr in CHR){
  print(paste0('-->',chr))
  clinvar_chr <- clinvar_def %>% filter(Chr == chr) %>% select(-Chr)
  
  if (length(pull(clinvar_chr, rsid)) == n_distinct(select(clinvar_chr, rsid))) {
    fwrite(clinvar_chr, file = paste0('/home/mentoraje/PAEs/SNProject/ClinVar/Resultats/ClinVarChr',chr,'.txt'), sep = '\t', dec ='.')
    print('OK')
  }
  
  else{print('NOT OK')} 
  
}

## DisGeNET 

In [ ]:
library (tidyr)
library(dplyr)
library(data.table)

#Load the raw data
dades <- fread('/home/mentoraje/PAEs/SNProject/Disgenet/all_variant_disease_associations.tsv')

CHR <- c(1:22)

#For each chromosome
for (chr in CHR){
  print(paste("-->", chr, sep=""))
  
  #Change the column name for future merging
  colnames(dades)[1] <- 'rsid'
  
  #Select the columns we are interested in and filter the SNPs by chromosome 
  dades_fltr <- dades %>% filter(dades$chromosome == chr) %>% distinct(rsid, .keep_all = TRUE) %>% select(rsid, chromosome, DSI, DPI, score, NofPmids) 
  
  #Summarize the values from repeated SNPs (are associated to more than 1 disease) keeping the maximum value from each column 
  summ <- dades_fltr %>% group_by(rsid) %>% summarise( max(DSI), 
                                                       max(DPI),
                                                       max(score), 
                                                       max(NofPmids))
  #Write the data in a different file for each chromosome
  fwrite(summ, file = paste0('/home/mentoraje/PAEs/SNProject/Disgenet/Resultats/chr',chr,'.txt'), sep = '\t', dec ='.')
  
}

## Atlas of Variant Age 

In [ ]:
library (tidyr)
library(dplyr)
library(data.table)

CHR <- c(1:22)

#For each chromosome 
for (chr in CHR) {
  print(paste0('-->', chr))
  
  #load the raw data
  Age <- fread(file = paste0('/home/mentoraje/PAEs/SNProject/Atlas_Variant_Age/DATA/atlas.chr',chr,'.csv'), header = TRUE, sep = ',') 
  
  #Change the column name to rsid for future merging 
  colnames(Age)[1] <- 'rsid'
  
  #Select those SNPs from the 1000GP source and the columns we are interested in
  Age_fltr <- Age %>% filter(Age$DataSource == 'TGP') %>% select(rsid, Position, AlleleAnc, starts_with('Age'), starts_with('Qual')) 
  
  #Write the data in a different file for each chromosome 
  fwrite(Age_fltr, file = paste0('/home/mentoraje/PAEs/SNProject/Atlas_Variant_Age/Resultats/chr',chr,'.txt'),sep = "\t")
  
}

## MERGE DisGeNet, ClinVar and RegulomeDB 

In [ ]:
library (tidyr)
library(dplyr)
library(data.table)


CHR <- c(1:22)
#For each chromosome
for (chr in CHR){
  print(paste0('-->',chr))
  
  #Load the filtered files from DisGeNET, ClinVar and RegulomeDB
  disgenet <- fread(paste0('/home/mentoraje/PAEs/SNProject/Disgenet/Resultats/chr',chr,'.txt'))
  clinvar <- fread(paste0('/home/mentoraje/PAEs/SNProject/ClinVar/Resultats/ClinVarChr',chr,'.txt'))
  regulome <- fread(paste0('/home/mentoraje/PAEs/SNProject/RegulomeDB/Resultats/RegulomeChr',chr,'.txt'))

  #Merge the three databases using full join (keep all the SNPs)   
  merge1 <- full_join(clinvar, regulome)
  merge2 <- full_join(merge1, disgenet)
  
  #Check the merge is correct
  rsid_disgenet <- pull(disgenet, rsid) 
  rsid_regulome <- pull(regulome, rsid)
  rsid_clinvar <- pull(clinvar, rsid)
  
  rsid_merge <- c(rsid_disgenet, rsid_regulome, rsid_clinvar)
  
  #If it is correct write a file per chromosome 
  if (n_distinct(rsid_merge) == length(pull(merge2, rsid))){
    print('OK')
    fwrite(merge2, file = paste0('/home/mentoraje/PAEs/SNProject/Merges/Merge_def/Resultats/MergeChr',chr,'.txt'), sep = '\t', dec ='.')
    
  }
  else{print('Not OK')}
   
}

## MERGE ALL 

In [ ]:
library (tidyr)
library(dplyr)
library(data.table)

CHR <- c(1:22)
for (chr in CHR){
  
  #Load the data from the previous merge (afegir), filtered age database (age) and 1000GP data with iHS, VEP and GWAS information (taula), previously merged by Daniel Soto 
  taula <- fread(paste0('/home/mentoraje/PAEs/SNProject/VEP/SNP_VEP_CHR',chr,'.txt'))
  afegir <- fread(paste0('/home/mentoraje/PAEs/SNProject/Merges/Merge_taula_general/Resultats_merge_1/MergeChr',chr,'.txt'), sep = '\t', dec ='.')
  age  <- fread(paste0('/home/mentoraje/PAEs/SNProject/Merges/Merge_taula_general/Resultats_age/chr',chr,'.txt'))
  
  #Merge 1000GP (+ iHS, GWAS, VEP) data with previously merged data
  taula_afegit <- left_join(taula, afegir)
  
  #Change the column name to physicalPos for future merging
  colnames(age)[2] <- 'physicalPos'
  
  #Remove the rsid column from age data to merge only by position
  age_sense_rsid <- age %>% select(!rsid)
  
  #Merge the previously merged data with age data
  taula_junt <- left_join(taula_afegit, age_sense_rsid)
  
  ##ADD MEAN, MEDIAN AND SD FROM iHS COLUMNS 
  #Select only the iHS columns
  dades_ihs <- taula_junt %>% select(starts_with('standiHS_'))
  
  #Convert the data to matrix format
  dades_ihs <- as.matrix(dades_ihs)
    
  #Calculate mean, median and sd for each row
  mean_calc <- apply(dades_ihs, 1, mean, na.rm = T)
  median_calc <- apply(dades_ihs, 1, median, na.rm = T)
  sd_calc <- apply(dades_ihs, 1, sd, na.rm = T)
  
  #Add the mean, median and sd columns
  dades_mean <- add_column(taula_junt, iHS_mean = mean_calc, .after = 'chr')
  dades_median <- add_column(dades_mean, iHS_median = median_calc, .after = 'iHS_mean')
  dades_final <- add_column(dades_median, iHS_sd = sd_calc, .after = 'iHS_median')
  
  #Write the data in a file for each chromosome
  fwrite(dades_final, paste0('/home/mentoraje/PAEs/SNProject/Merges/Merge_taula_general/Resultats/Taula_def_chr_',chr,'.txt'),  sep = '\t', dec ='.')

}

##Merge all the chromosomes 
#load the final data from chromosome 1
taula_inicial <- fread('/home/mentoraje/PAEs/SNProject/Merges/Merge_taula_general/Resultats/Taula_def_chr_1.txt')
ADD <- c(2:22)
#for each chromosome (but chr1) load the data from the chromosome variants and add them to the final table
for (x in ADD){
  taula_afegir <- fread(paste0('/home/mentoraje/PAEs/SNProject/Merges/Merge_taula_general/Resultats/Taula_def_chr_',chr,'.txt'))
  taula_inicial <- bind_rows(taula_inicial, taula_afegir)
}

#Write the final data in a file
fwrite(taula_inicial, '/home/mentoraje/PAEs/SNProject/Merges/Merge_taula_general/Resultats/TAULA_DEFINITIVA.txt')

